### UP 2010 Clean up

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
up_2010 = pd.read_csv("../data/up_gp_sarpanch_2010.csv", low_memory = False)
up_2010.head()

,original_filename,page,order,district_name,sr_no,gp_code,block_code,block_name,sr_gp_combo,gp_name,gp_reservation_status,elected_sarpanch_name,husband_spouse_name,candidate_res_status,age,sex,education,address
0,gpp_0503.pdf,22,6193,मुरादाबाद,544,28,8,बहजोई,544 28,खजरा खाकम,अनारक्षित,पिंकी कुमारी,कुमारी मुनेश कुमार,कुमार अनारक्षित,32,महिला,इण्टर,ग्राम भोजपुर पो ० नाधौस
1,gpp_0602.pdf,25,9018,पीलीभीत,596,153,7,पूरनपुर,596,सिमराया ता ० घुंघचिह,पिछड़ी जाति,जगदेव सिंह,अनोख सिंह,पिछड़ी जाति,29,पुरुष,स्नातक,सिमराया ता O घुघचिहाई
2,gpp_0301.pdf,8,223,सहारनपुर,204,33,4,नानौता,204 33,इन्द्राबली कल्याणपुर,अनारक्षित,रेखा,योगेन्द्र सिंह,अनुसूचित जाति महिला,32,महिला,जूनियर हाई स्कूल,इन्द्रावली पो ० जांडखेडा
3,gpp_1301.pdf,9,33350,बहराइच,232,NaN,4,फखरपुर,232 35,घरहेरीपुर,अनारक्षित,सामान अली,नवाब अली,अनुसूचित जाति महिला,55,पुरुष,साक्षर,ग्राम व पो ० घरेहरीपुर
4,gpp_1401.pdf,40,37391,बाराबंकी,969,NaN,15,सिरौलीगौसपुर,969 32,बरदरी,अनारक्षित,राजेश्वरी,प्राणपती,अनुसूचित जाति महिला,50 महिला,NaN,साक्षर,बरदरी पो . मरकामऊ


### Let's get GP Name sorted

In [3]:
up_2010['gp_name_temp'] = up_2010['sr_gp_combo'].str.strip().str.replace('[0-9\[\]]', '', regex=True)

In [4]:
### Normalize GP Name

up_2010['gp_name'] = up_2010['gp_name'].str.strip()\
                        .str.replace(r'^(\s?-\s?|\s?--\s?)|(\s-|\s--|\s---)$', '', regex=True)\
                        .str.replace(r'\s{2,}', ' ', regex=True)\
                        .str.strip()\
                        .str.replace('[0-9\[\]]', '', regex=True)

# Let's take out reservation status from GP Name
gp_res_status = [
    'अनारक्षित',
    'महिला',
    'अनुसूचित जाति - महिला',
    'पिछड़ी जाति - महिला',
    'पिछड़ी जाति',
    'अनुसूचित जाति',
    'पिछड़ी जाति महिला -',
    'अनुसूचित जाति महिला -'
    'महिला'
    'अनुसूचित जाति महिला -',
    'पिछड़ी जाति महिला',
    'अनुसूचित जाति महिला',
    'पिछड़ी जाति महिला -'
    'अनुसूचित जाति महिला -',
    'पिछड़ी जाति महिला --',
    'अनुसूचित जाति महिला - '
]
up_2010['gp_res_status_temp'] = np.where(up_2010['gp_name'].str.strip().isin(gp_res_status), up_2010['gp_name'], '')
up_2010['gp_name'] = np.where(~up_2010['gp_name'].isin(gp_res_status), up_2010['gp_name'], '')
up_2010['gp_name'].value_counts()[:20]

रामपुर        52
रसूलपुर       50
गोपालपुर      49
रामनगर        42
शाहपुर        41
रायपुर        41
सलेमपुर       39
जगदीशपुर      36
सिकन्दरपुर    36
गौरा          36
भगवानपुर      35
जलालपुर       35
नरायनपुर      34
कल्यानपुर     32
दौलतपुर       31
बहादुरपुर     31
बरौली         30
सुल्तानपुर    29
खानपुर        28
मिर्जापुर     27
Name: gp_name, dtype: int64

In [5]:
up_2010['gp_name_fin'] = up_2010.apply(lambda x: x['gp_name_temp'] if x['gp_name_temp'] != '' 
                            else x['gp_name'], axis=1)
up_2010['gp_name_fin']

0                            
1        सिमराया ता ० घुंघचिह
2                            
3                            
4                            
                 ...         
51856          खुदिया बुजुर्ग
51857                    परोई
51858                  रामनगर
51859                        
51860            सिरसिया पवार
Name: gp_name_fin, Length: 51861, dtype: object

In [6]:
up_2010['gp_reservation_status'].value_counts()[:30]

अनारक्षित                                      17925
पिछड़ी जाति                                     9473
महिला                                           8401
अनुसूचित जाति                                   7168
पिछड़ी जाति - महिला                             2482
पिछड़ी जाति महिला                               2343
अनुसूचित जाति - महिला                           2106
अनुसूचित जाति महिला                             1718
पिछडी जाति                                        64
पिछडी जाति - महिला                                53
पिछड़ी जाति महिला                                  37
अनुसूचित जन जाति                                  17
अनुसूचित जन जाति - महिला                          14
पिछड़ी जाति - महिला                                 7
पिछडी जाति महिला                                   6
अनारक्षित                                          3
पिछड़ी जाति                                        2
 अनारक्षित                                         2
पिछड़ी जाति - महिला -                         

In [7]:
up_2010['gp_res_status_fin'] = up_2010.apply(lambda x: x['gp_res_status_temp'] if x['gp_res_status_temp'] != '' 
                            else x['gp_reservation_status'], axis=1)
up_2010['gp_res_status_fin'].value_counts()[:24]

अनारक्षित                                      17924
पिछड़ी जाति                                     9473
महिला                                           8402
अनुसूचित जाति                                   7168
पिछड़ी जाति - महिला                             2482
पिछड़ी जाति महिला                               2343
अनुसूचित जाति - महिला                           2106
अनुसूचित जाति महिला                             1718
पिछडी जाति                                        64
पिछडी जाति - महिला                                53
पिछड़ी जाति महिला                                  37
अनुसूचित जन जाति                                  17
अनुसूचित जन जाति - महिला                          14
पिछड़ी जाति - महिला                                 7
पिछडी जाति महिला                                   6
अनारक्षित                                          3
पिछड़ी जाति                                        2
 अनारक्षित                                         2
पिछड़ी जाति - महिला -                         

In [8]:
# Normalize
up_2010['gp_res_status_fin'] = (up_2010['gp_res_status_fin']
                 .str.strip()  # Step 1: Trim leading and trailing spaces   
                 .str.replace(r'^(\s?-\s?|\s?--\s?)|(\s-|\s--|\s---)$', '', regex=True)  # Step 2: Remove specific patterns at the start/end
                 .str.strip()  # Step 1: Trim leading and trailing spaces
                 .str.replace(r'\s{2,}', ' ', regex=True).str.strip())  # Step 3: Replace multiple spaces with a single space
up_2010['gp_res_status_fin'].value_counts()[:15]

अनारक्षित                   17929
पिछड़ी जाति                  9476
महिला                        8402
अनुसूचित जाति                7169
पिछड़ी जाति - महिला          2485
पिछड़ी जाति महिला            2343
अनुसूचित जाति - महिला        2106
अनुसूचित जाति महिला          1720
पिछडी जाति                     64
पिछडी जाति - महिला             53
पिछड़ी जाति महिला               37
अनुसूचित जन जाति               17
अनुसूचित जन जाति - महिला       14
पिछड़ी जाति - महिला              7
पिछडी जाति महिला                6
Name: gp_res_status_fin, dtype: int64

In [9]:
# Define the list of substrings to check for
valid_categories = up_2010['gp_res_status_fin'].value_counts()[:15].index.to_list()

up_2010['gp_res_status_fin'] = np.where(up_2010['gp_res_status_fin'].isin(valid_categories), up_2010['gp_res_status_fin'], '')
up_2010['gp_res_status_fin'].value_counts()[:20]

अनारक्षित                   17929
पिछड़ी जाति                  9476
महिला                        8402
अनुसूचित जाति                7169
पिछड़ी जाति - महिला          2485
पिछड़ी जाति महिला            2343
अनुसूचित जाति - महिला        2106
अनुसूचित जाति महिला          1720
पिछडी जाति                     64
पिछडी जाति - महिला             53
पिछड़ी जाति महिला               37
                               33
अनुसूचित जन जाति               17
अनुसूचित जन जाति - महिला       14
पिछड़ी जाति - महिला              7
पिछडी जाति महिला                6
Name: gp_res_status_fin, dtype: int64

In [10]:
translit_dict = {
    'अनारक्षित': 'Unreserved',
    'पिछड़ी जाति': 'Other Backward Class',
    'महिला': 'Female',
    'अनुसूचित जाति': 'Scheduled Caste',
    'पिछड़ी जाति - महिला': 'Other Backward Class - Female',
    'पिछड़ी जाति महिला': 'Other Backward Class - Female',
    'अनुसूचित जाति - महिला': 'Scheduled Caste - Female',
    'अनुसूचित जाति महिला': 'Scheduled Caste - Female',
    'पिछडी जाति': 'Other Backward Class',
    'पिछडी जाति - महिला': 'Other Backward Class - Female',
    'पिछड़ी जाति महिला': 'Other Backward Class - Female',
    '': 'Unknown',  
    'अनुसूचित जन जाति': 'Scheduled Tribe',
    'अनुसूचित जन जाति - महिला': 'Scheduled Tribe - Female',
    'पिछड़ी जाति - महिला': 'Other Backward Class - Female',
    'पिछडी जाति महिला': 'Other Backward Class - Female'
}

up_2010['gp_res_status_fin_eng'] = up_2010['gp_res_status_fin'].map(translit_dict)
up_2010['gp_res_status_fin_eng'].value_counts()

Unreserved                       17929
Other Backward Class              9540
Female                            8402
Scheduled Caste                   7169
Other Backward Class - Female     4931
Scheduled Caste - Female          3826
Unknown                             33
Scheduled Tribe                     17
Scheduled Tribe - Female            14
Name: gp_res_status_fin_eng, dtype: int64

In [11]:
def process_info(info):
    if not isinstance(info, str):
        return "", "", None  # Handle non-string data directly

    # Initialize default values for gender and age
    gender = ""
    age = None

    keywords = {'महिला': 'woman', 'पुरुष': 'man'}
    age_pattern = re.compile(r'\b\d+\b')  # Matches whole numbers

    # Extract age from the string (assuming the first number found is the age)
    found_age = age_pattern.findall(info)
    if found_age:
        age = int(found_age[0])  # Convert the first found age to integer
        info = age_pattern.sub("", info).strip()  # Remove the age from the string

    # Check for and process keywords
    for keyword, eng_gender in keywords.items():
        if keyword in info:
            gender = keyword #eng_gender
            info = info.replace(keyword, '').strip()  # Strip keyword and any extra spaces

    return info, gender, age

### Split Cols on the Right

In [12]:
up_2010['candidate_res_status_fin'], up_2010['cand_sex_t'], up_2010['age_t'] = zip(*up_2010['candidate_res_status'].apply(process_info))
up_2010['candidate_res_status_fin'].value_counts()

पिछड़ी जाति                        22555
अनुसूचित जाति                      10586
अनारक्षित                          10078
पिछड़ी जाति -                       3434
                                    2577
अनुसूचित जाति -                      983
अनुसूचित जाति  -                     912
पिछडी जाति                           342
- अनुसूचित जाति                       86
अनुसूचित जन जाति                      60
पिछड़ी जाति                            59
अनुसूचित जा                           52
- पिछड़ी जाति                         48
अनुसूचित जाि                          15
अनुसूचित जन जाति -                    14
अनुसूचित जात                          10
अनुसूचित जाति  --                      6
- पिछड़ी जाति                           6
पिछड़ी जाति\nअनारक्षित                 3
पिछड़ी जाति\nपिछड़ी जाति               2
-- अनुसूचित जाति                       2
पिछड़ी जाति -                           2
अनुसूचित जाति\nपिछड़ी जाति             2
अनारक्षित\nपिछड़ी जाति                 2
अनुसूचित जाति\nअ

In [13]:
up_2010['educ_t2'], up_2010['cand_sex_t2'], up_2010['age_t2'] = zip(*up_2010['sex'].apply(process_info))
up_2010['cand_sex_t2'].value_counts()

         37092
पुरुष     8497
महिला     6272
Name: cand_sex_t2, dtype: int64

In [14]:
up_2010['educ_t3'], up_2010['cand_sex_t3'], up_2010['age_t3'] = zip(*up_2010['age'].apply(process_info))
up_2010['cand_sex_t3'].value_counts()

पुरुष    21690
महिला    15385
         14786
Name: cand_sex_t3, dtype: int64

In [15]:
up_2010['educ_t4'], up_2010['cand_sex_t4'], up_2010['age_t4'] = zip(*up_2010['education'].apply(process_info))
up_2010['educ_t4'].value_counts()

                                     20784
साक्षर                                6456
जूनियर हाई स्कूल                      5229
इण्टर                                 4849
हाई स्कूल                             4546
प्राईमरी                              3684
स्नातक                                3460
निरक्षर                               1628
परास्नातक                             1207
जूनियर हाई स्कूल जूनियर हाई स्कूल        2
इण्टर \nप्राईमरी                         2
जूनियर हाई स्कूल \nहाई स्कूल             2
हाई स्कूल                                1
मटेहॅू                                   1
परास्नातक \nहाई स्कूल                    1
प्राईमरी \nजूनियर हाई स्कूल              1
जूनियर हाई स्कूल \nप्राईमरी              1
जूनियर हाई स्कूल \nसाक्षर                1
हाई स्कूल \nसाक्षर                       1
हाई स्कूल ग्राम\n                        1
साक्षर \nहाई स्कूल                       1
साक्षर जूनियर हाई स्कूल                  1
इण्टर \nजूनियर हाई स्कूल                 1
परास्नातक  

### Finalize

In [16]:
# Candidate Sex Final
up_2010['cand_sex_fin'] = up_2010.apply(lambda x: x['cand_sex_t2'] if x['cand_sex_t3'] == '' else x['cand_sex_t3'], axis=1)
up_2010['cand_sex_fin'].value_counts()

पुरुष    30187
महिला    21656
            18
Name: cand_sex_fin, dtype: int64

In [17]:
# Candidate Age Final
up_2010['age_fin'] = up_2010.apply(lambda x: x['age_t2'] if x['age_t3'] == '' else x['age_t3'], axis=1)
up_2010['age_fin'].isna().sum()

154

In [18]:
up_2010['age_fin'].describe()

count    51707.000000
mean        41.192546
std          9.958541
min          5.000000
25%         35.000000
50%         40.000000
75%         48.000000
max        105.000000
Name: age_fin, dtype: float64

In [19]:
up_2010['educ_t3'].value_counts()

                                           31356
साक्षर                                      6994
हाई स्कूल                                   3118
प्राईमरी                                    3016
स्नातक                                      2106
इण्टर                                       2098
जूनियर हाई स्कूल                            1312
निरक्षर                                     1160
परास्नातक                                    664
जूनियर हाई                                     3
निरक्षर\nप्राईमरी                              3
स्नातक\n  इण्टर                                2
जूनियर हाई स्कूल जूनियर हाई स्कूल              2
प्राईमरी\nनिरक्षर                              2
इण्टर\n  साक्षर                                1
परास्नातक\n  जूनियर हाई स्कूल                  1
परास्नातक  \n  स्नातक                          1
साक्षर\n साक्षर                                1
परास्नातक\n प्राईमरी                           1
स्नातक\n साक्षर                                1
स्नातक\nनिरक्षर     

In [20]:
# Candidate Education
up_2010['educ_fin'] = up_2010.apply(lambda x: x['educ_t2'] if x['educ_t3'] == '' else x['educ_t3'], axis=1)
up_2010['educ_fin'] = up_2010.apply(lambda x: x['educ_fin'] if x['educ_t4'] == '' else x['educ_t4'], axis=1)
up_2010['educ_fin'].value_counts()

साक्षर                                     13453
हाई स्कूल                                   7668
इण्टर                                       6951
प्राईमरी                                    6748
जूनियर हाई स्कूल                            6545
स्नातक                                      5570
निरक्षर                                     2789
परास्नातक                                   1871
                                             211
जूनियर हाई स्कूल जूनियर हाई स्कूल              4
जूनियर हाई                                     3
निरक्षर\nप्राईमरी                              3
स्नातक\n  इण्टर                                2
प्राईमरी\nनिरक्षर                              2
इण्टर \nप्राईमरी                               2
प्राईमरी\n  जूनियर हाई स्कूल                   2
जूनियर हाई स्कूल \nहाई स्कूल                   2
हाई स्कूल ग्राम\n                              1
निरक्षर\nनिरक्षर                               1
परास्नातक\n  जूनियर हाई स्कूल                  1
इण्टर\n  साक्षर     

In [21]:
translit_dict = {
    'साक्षर': 'Literate',
    'हाई स्कूल': 'High School',
    'इण्टर': 'Intermediate',
    'प्राईमरी': 'Primary',
    'जूनियर हाई स्कूल': 'Junior High School',
    'स्नातक': 'Graduate',
    'निरक्षर': 'Illiterate',
    'परास्नातक': 'Postgraduate'
}

up_2010['educ_fin_eng'] = up_2010['educ_fin'].map(translit_dict)
up_2010['educ_fin_eng'].value_counts()

Literate              13453
High School            7668
Intermediate           6951
Primary                6748
Junior High School     6545
Graduate               5570
Illiterate             2789
Postgraduate           1871
Name: educ_fin_eng, dtype: int64

In [22]:
up_2010.columns

Index(['original_filename', 'page', 'order', 'district_name', 'sr_no',
       'gp_code', 'block_code', 'block_name', 'sr_gp_combo', 'gp_name',
       'gp_reservation_status', 'elected_sarpanch_name', 'husband_spouse_name',
       'candidate_res_status', 'age', 'sex', 'education', 'address',
       'gp_name_temp', 'gp_res_status_temp', 'gp_name_fin',
       'gp_res_status_fin', 'gp_res_status_fin_eng',
       'candidate_res_status_fin', 'cand_sex_t', 'age_t', 'educ_t2',
       'cand_sex_t2', 'age_t2', 'educ_t3', 'cand_sex_t3', 'age_t3', 'educ_t4',
       'cand_sex_t4', 'age_t4', 'cand_sex_fin', 'age_fin', 'educ_fin',
       'educ_fin_eng'],
      dtype='object')

In [23]:
up_2010.drop(columns =['gp_name_temp', 'gp_res_status_temp',
              'cand_sex_t', 'age_t', 
              'educ_t2', 'cand_sex_t2', 'age_t2', 
              'cand_sex_t3', 'age_t3', 'educ_t3',
              'cand_sex_t4', 'age_t4', 'educ_t4'], inplace = True)
up_2010.columns

Index(['original_filename', 'page', 'order', 'district_name', 'sr_no',
       'gp_code', 'block_code', 'block_name', 'sr_gp_combo', 'gp_name',
       'gp_reservation_status', 'elected_sarpanch_name', 'husband_spouse_name',
       'candidate_res_status', 'age', 'sex', 'education', 'address',
       'gp_name_fin', 'gp_res_status_fin', 'gp_res_status_fin_eng',
       'candidate_res_status_fin', 'cand_sex_fin', 'age_fin', 'educ_fin',
       'educ_fin_eng'],
      dtype='object')

In [24]:
up_2010.to_csv("../data/up_gp_sarpanch_2010_fixed.csv", index = False)